# Despliegue de Modelo - Evaluación Conjunta

Lo primero que haremos, será el ver que es lo que contiene nuestro dataset

In [21]:
# Importamos nuestras librerías

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
def leer_datos(ruta):
    df = pd.read_csv(ruta,sep=',')
    return df

In [6]:
entrenamiento = leer_datos("data_evaluacion.csv")
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48841 entries, 0 to 48840
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   39             48841 non-null  int64 
 1   State-gov      48841 non-null  object
 2   77516          48841 non-null  int64 
 3   Bachelors      48841 non-null  object
 4   13             48841 non-null  int64 
 5   Never-married  48841 non-null  object
 6   Adm-clerical   48841 non-null  object
 7   Not-in-family  48841 non-null  object
 8   White          48841 non-null  object
 9   Male           48841 non-null  object
 10  2174           48841 non-null  int64 
 11  0              48841 non-null  int64 
 12  40             48841 non-null  int64 
 13  United-States  48841 non-null  object
 14  <=50K          48841 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


Como vemos, nuestro dataset consta de 48841 datos, sin embargo, colocaremos encabezados basados en la información de nuestro dataset, para que nos sea más fácil entender las columnas que estamos manejando.

In [7]:
# Cargamos el archivo CSV de la evaluacion
file_path = 'data_evaluacion.csv'
df = pd.read_csv(file_path, header=None)  # header=None indica que el archivo CSV no tiene encabezado

# Asignamos títulos a las columnas
df.columns = ['Edad', 'ClaseObrera', 'PesoFinal', 'Educacion', 'EducacionNum', 'EstadoMarital', 'Ocupacion', 'EstadoCivil', 'Raza', 'Sexo', 'GananciaCapital'
, 'PerdidaCapital', 'HorasPorSemana', 'Pais', 'Salario']

# Guardamos el DataFrame con los nuevos títulos de columna
df.to_csv('data_evaluacion_encabezados.csv', index=False)

print("Archivo CSV con encabezados guardado con éxito.")

Archivo CSV con encabezados guardado con éxito.


In [46]:
# Volvemos a ver si nuestro nuevo archivo tiene los mismos datos

df = leer_datos("data_evaluacion_encabezados.csv")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Edad             48842 non-null  int64 
 1   ClaseObrera      48842 non-null  object
 2   PesoFinal        48842 non-null  int64 
 3   Educacion        48842 non-null  object
 4   EducacionNum     48842 non-null  int64 
 5   EstadoMarital    48842 non-null  object
 6   Ocupacion        48842 non-null  object
 7   EstadoCivil      48842 non-null  object
 8   Raza             48842 non-null  object
 9   Sexo             48842 non-null  object
 10  GananciaCapital  48842 non-null  int64 
 11  PerdidaCapital   48842 non-null  int64 
 12  HorasPorSemana   48842 non-null  int64 
 13  Pais             48842 non-null  object
 14  Salario          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [19]:
# Verificamos si hay valores nulos en las columnas

print(df.isnull().sum())

Edad               0
ClaseObrera        0
PesoFinal          0
Educacion          0
EducacionNum       0
EstadoMarital      0
Ocupacion          0
EstadoCivil        0
Raza               0
Sexo               0
GananciaCapital    0
PerdidaCapital     0
HorasPorSemana     0
Pais               0
Salario            0
dtype: int64


## Limpieza de Datos

In [30]:
# Convertimos variables categóricas a numéricas
le = LabelEncoder()
categorical_columns = ['ClaseObrera', 'Educacion', 'EstadoMarital', 'Ocupacion', 'EstadoCivil', 'Raza', 'Sexo', 'Pais', 'Salario']
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

# Separamos características y variable objetivo
X = df.drop('Salario', axis=1)
y = df['Salario']

# Dividimos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalamos las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
# NORMALIZACION DE CARACTERISTICAS NUMERICAS
scaler = StandardScaler()
numeric_columns = X.select_dtypes(include=[np.number]).columns
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test_scaled[numeric_columns] = scaler.transform(X_test[numeric_columns])


## Entrenamiento de los Modelos

In [31]:
# Función para entrenar y evaluar modelos
def train_evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} - Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))
    return model, accuracy

# Regresión Logística
lr_model, lr_accuracy = train_evaluate_model(LogisticRegression(random_state=42), X_train_scaled, X_test_scaled, y_train, y_test, "Logistic Regression")

# SVM (4 kernels)
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
svm_models = {}
for kernel in kernels:
    svm_model, svm_accuracy = train_evaluate_model(SVC(kernel=kernel, random_state=42), X_train_scaled, X_test_scaled, y_train, y_test, f"SVM ({kernel})")
    svm_models[kernel] = (svm_model, svm_accuracy)

# K-NN (Identificando el mejor número para k)
k_values = range(1, 21)
knn_accuracies = []
for k in k_values:
    knn_model, knn_accuracy = train_evaluate_model(KNeighborsClassifier(n_neighbors=k), X_train_scaled, X_test_scaled, y_train, y_test, f"KNN (k={k})")
    knn_accuracies.append(knn_accuracy)

best_k = k_values[np.argmax(knn_accuracies)]
print(f"Mejor valor de k: {best_k}")

# Árboles de Decisión
dt_model, dt_accuracy = train_evaluate_model(DecisionTreeClassifier(random_state=42), X_train_scaled, X_test_scaled, y_train, y_test, "Decision Tree")

# Naive Bayes
nb_model, nb_accuracy = train_evaluate_model(GaussianNB(), X_train_scaled, X_test_scaled, y_train, y_test, "Naive Bayes")

# Redes Neuronales
nn_model, nn_accuracy = train_evaluate_model(MLPClassifier(random_state=42), X_train_scaled, X_test_scaled, y_train, y_test, "Neural Network")

Logistic Regression - Accuracy: 0.8237281195618794
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      7414
           1       0.72      0.44      0.55      2355

    accuracy                           0.82      9769
   macro avg       0.78      0.69      0.72      9769
weighted avg       0.81      0.82      0.81      9769

SVM (linear) - Accuracy: 0.8127751049237384
              precision    recall  f1-score   support

           0       0.81      0.98      0.89      7414
           1       0.79      0.30      0.44      2355

    accuracy                           0.81      9769
   macro avg       0.80      0.64      0.66      9769
weighted avg       0.81      0.81      0.78      9769

SVM (poly) - Accuracy: 0.8416419285494933
              precision    recall  f1-score   support

           0       0.86      0.95      0.90      7414
           1       0.75      0.51      0.61      2355

    accuracy                           0.84  

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [32]:
# Recopilar accuracies
accuracies = {
    'Logistic Regression': lr_accuracy,
    'SVM': max(svm_models.values(), key=lambda x: x[1])[1],
    'KNN': max(knn_accuracies),
    'Decision Tree': dt_accuracy,
    'Naive Bayes': nb_accuracy,
    'Neural Network': nn_accuracy
}

best_model = max(accuracies, key=accuracies.get)
print(f"El mejor modelo es: {best_model} con una precisión de {accuracies[best_model]}")

El mejor modelo es: Neural Network con una precisión de 0.8514689323369843


In [33]:
# Identificamos Overfiting o Underfiting
def check_fit(model, X_train, X_test, y_train, y_test, model_name):
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    print(f"{model_name}:")
    print(f"  Train Accuracy: {train_accuracy}")
    print(f"  Test Accuracy: {test_accuracy}")
    if train_accuracy > test_accuracy + 0.05:
        print("  Posible overfitting")
    elif test_accuracy > train_accuracy + 0.05:
        print("  Posible underfitting")
    else:
        print("  Buen ajuste")

# Verificar ajuste para cada modelo
check_fit(lr_model, X_train_scaled, X_test_scaled, y_train, y_test, "Logistic Regression")
for kernel, (model, _) in svm_models.items():
    check_fit(model, X_train_scaled, X_test_scaled, y_train, y_test, f"SVM ({kernel})")
check_fit(KNeighborsClassifier(n_neighbors=best_k).fit(X_train_scaled, y_train), X_train_scaled, X_test_scaled, y_train, y_test, "KNN")
check_fit(dt_model, X_train_scaled, X_test_scaled, y_train, y_test, "Decision Tree")
check_fit(nb_model, X_train_scaled, X_test_scaled, y_train, y_test, "Naive Bayes")
check_fit(nn_model, X_train_scaled, X_test_scaled, y_train, y_test, "Neural Network")

Logistic Regression:
  Train Accuracy: 0.8251989865124255
  Test Accuracy: 0.8237281195618794
  Buen ajuste
SVM (linear):
  Train Accuracy: 0.8136564891357203
  Test Accuracy: 0.8127751049237384
  Buen ajuste
SVM (poly):
  Train Accuracy: 0.8487702505566503
  Test Accuracy: 0.8416419285494933
  Buen ajuste
SVM (rbf):
  Train Accuracy: 0.8566273385713921
  Test Accuracy: 0.8456341488381616
  Buen ajuste
SVM (sigmoid):
  Train Accuracy: 0.7505438538120953
  Test Accuracy: 0.7548367284266557
  Buen ajuste
KNN:
  Train Accuracy: 0.8526859980037366
  Test Accuracy: 0.8341693110860886
  Buen ajuste
Decision Tree:
  Train Accuracy: 0.9999232206382924
  Test Accuracy: 0.8107278124680111
  Posible overfitting
Naive Bayes:
  Train Accuracy: 0.8036239858725974
  Test Accuracy: 0.8028457365134609
  Buen ajuste
Neural Network:
  Train Accuracy: 0.8662247587848386
  Test Accuracy: 0.8514689323369843
  Buen ajuste


In [82]:
import joblib

# Guardar el mejor modelo y el scaler
joblib.dump(best_model, 'best_model.joblib')
joblib.dump(scaler, 'scaler.joblib')

['scaler.joblib']

In [84]:
# Guardamos el mejor modelo

import joblib

# Identificar el mejor modelo
accuracies = {
    'Logistic Regression': lr_accuracy,
    'SVM': max(svm_models.values(), key=lambda x: x[1])[1],
    'KNN': max(knn_accuracies),
    'Decision Tree': dt_accuracy,
    'Naive Bayes': nb_accuracy,
    'Neural Network': nn_accuracy
}
best_model_name = max(accuracies, key=accuracies.get)

# Obtener la instancia del mejor modelo
if best_model_name == 'Logistic Regression':
    best_model = lr_model
elif best_model_name.startswith('SVM'):
    best_kernel = max(svm_models, key=lambda x: svm_models[x][1])
    best_model = svm_models[best_kernel][0]
elif best_model_name == 'KNN':
    best_model = KNeighborsClassifier(n_neighbors=best_k)
    best_model.fit(X_train_scaled, y_train)
elif best_model_name == 'Decision Tree':
    best_model = dt_model
elif best_model_name == 'Naive Bayes':
    best_model = nb_model
elif best_model_name == 'Neural Network':
    best_model = nn_model

# Guardar el mejor modelo
joblib.dump(best_model, 'mejor_modelo.joblib')

# Guardar el scaler
joblib.dump(scaler, 'scaler.joblib')

# Guardar los LabelEncoders
encoders = {}
for col in categorical_columns:
    encoders[col] = LabelEncoder()
    encoders[col].fit(df[col])
joblib.dump(encoders, 'label_encoders.joblib')

# Guardar el orden de las columnas
joblib.dump(X.columns.tolist(), 'column_order.joblib')

print(f"El mejor modelo ({best_model_name}) ha sido guardado.")

El mejor modelo (Neural Network) ha sido guardado.


In [95]:
# Función de Búsqueda

import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def predict_salary(data):
    # Cargar el modelo, scaler y encoders
    model = joblib.load('mejor_modelo.joblib')
    scaler = joblib.load('scaler.joblib')
    encoders = joblib.load('label_encoders.joblib')

    # Definir las columnas categóricas (asegúrate de tener esta lista)
    categorical_columns = ['ClaseObrera', 'Educacion', 'EstadoMarital', 'Ocupacion', 'EstadoCivil', 'Raza', 'Sexo', 'Pais']

    # Crear una copia de los datos para no modificar el original
    data_encoded = data.copy()

    # Aplicar Label Encoding a las columnas categóricas, manejando valores desconocidos
    for col in categorical_columns:
        if col in data_encoded.columns:
            if col not in encoders:
                print(f"Warning: No encoder found for column {col}. Skipping encoding for this column.")
                continue
            encoder = encoders[col]
            # Handle unknown values by setting them to a placeholder (e.g., 'Unknown')
            data_encoded[col] = data_encoded[col].map(lambda s: s if s in encoder.classes_ else 'Unknown')
            # Add 'Unknown' to the encoder classes if it's not already there
            if 'Unknown' not in encoder.classes_:
                encoder.classes_ = np.append(encoder.classes_, 'Unknown')
            data_encoded[col] = encoder.transform(data_encoded[col][data_encoded[col].notna()])

    # Asegurarse de que las columnas estén en el mismo orden que durante el entrenamiento
    column_order = joblib.load('column_order.joblib')
    missing_cols = set(column_order) - set(data_encoded.columns)
    for col in missing_cols:
        data_encoded[col] = 0  # o cualquier otro valor por defecto
    data_encoded = data_encoded.reindex(columns=column_order)

    # Convertir las columnas a float64 y llenar NaNs con 0
    for col in data_encoded.columns:
        data_encoded[col] = pd.to_numeric(data_encoded[col], errors='coerce')
        data_encoded[col] = data_encoded[col].fillna(0)  # Reemplazar NaNs con 0

    # Escalar los datos
    X = scaler.transform(data_encoded)

    # Hacer la predicción
    prediction = model.predict(X)

    # Verificar si el modelo tiene el método predict_proba
    if hasattr(model, 'predict_proba'):
        probability = model.predict_proba(X)[:, 1]
    else:
        probability = None  # Para modelos que no proporcionan probabilidades

    return prediction, probability

# Ejemplo de uso
new_data = pd.DataFrame({
    'Edad': [31],
    'ClaseObrera': ['Private'],
    'PesoFinal': [84154],
    'Educacion': ['Masters'],
    'EducacionNum': [14],
    'EstadoMarital': ['Never-married'],
    'Ocupacion': ['Sales'],
    'EstadoCivil': ['Husband'],
    'Raza': ['White'],
    'Sexo': ['Male'],
    'GananciasCapital': [14084],
    'PerdidasCapital': [0],
    'HorasPorSemana': [80],
    'Pais': ['Cuba']
})

prediction, probability = predict_salary(new_data)
print(f"Predicted salary: {'<=50K' if prediction[0] == 0 else '>50K'}")
if probability is not None:
    print(f"Probability of salary >50K: {probability[0]:.2f}")
else:
    print("Probability not available for this model.")

Predicted salary: >50K
Probability of salary >50K: 0.82


Función de Predicción Mejorada

Únicamente utilizada para comprobar que se está prediciendo

In [94]:
import joblib
import pandas as pd
import numpy as np

def predict_salary(data):
    # Cargar el modelo, scaler y encoders
    model = joblib.load('mejor_modelo.joblib')
    scaler = joblib.load('scaler.joblib')
    encoders = joblib.load('label_encoders.joblib')

    # Definir las columnas categóricas
    categorical_columns = ['ClaseObrera', 'Educacion', 'EstadoMarital', 'Ocupacion', 'EstadoCivil', 'Raza', 'Sexo', 'Pais']

    # Crear una copia de los datos para no modificar el original
    data_encoded = data.copy()

    # Aplicar Label Encoding a las columnas categóricas, manejando valores desconocidos
    for col in categorical_columns:
        if col in data_encoded.columns:
            if col not in encoders:
                print(f"Warning: No encoder found for column {col}. Skipping encoding for this column.")
                continue
            encoder = encoders[col]
            # Manejar valores desconocidos
            data_encoded[col] = data_encoded[col].apply(lambda x: x if x in encoder.classes_ else 'Unknown')
            # Asegurarse de que 'Unknown' esté en las clases del encoder
            if 'Unknown' not in encoder.classes_:
                encoder.classes_ = np.append(encoder.classes_, 'Unknown')
            # Transformar los datos
            data_encoded[col] = encoder.transform(data_encoded[col])

    # Asegurarse de que las columnas estén en el mismo orden que durante el entrenamiento
    column_order = joblib.load('column_order.joblib')
    missing_cols = set(column_order) - set(data_encoded.columns)
    for col in missing_cols:
        data_encoded[col] = 0  # o cualquier otro valor por defecto
    data_encoded = data_encoded.reindex(columns=column_order)

    # Convertir las columnas a float64 y llenar NaNs con 0
    for col in data_encoded.columns:
        data_encoded[col] = pd.to_numeric(data_encoded[col], errors='coerce')
        data_encoded[col] = data_encoded[col].fillna(0)  # Reemplazar NaNs con 0

    # Escalar los datos
    X = scaler.transform(data_encoded)

    # Hacer la predicción
    prediction = model.predict(X)

    # Verificar si el modelo tiene el método predict_proba
    if hasattr(model, 'predict_proba'):
        probability = model.predict_proba(X)[:, 1]
    else:
        probability = None  # Para modelos que no proporcionan probabilidades

    return prediction, probability

# Ejemplo de uso
new_data = pd.DataFrame({
    'Edad': [31],
    'ClaseObrera': ['Private'],
    'PesoFinal': [84154],
    'Educacion': ['Masters'],
    'EducacionNum': [14],
    'EstadoMarital': ['Never-married'],
    'Ocupacion': ['Sales'],
    'EstadoCivil': ['Husband'],
    'Raza': ['White'],
    'Sexo': ['Male'],
    'GananciasCapital': [14084],
    'PerdidasCapital': [0],
    'HorasPorSemana': [80],
    'Pais': ['Cuba']
})

prediction, probability = predict_salary(new_data)
print(f"Predicted salary: {'<=50K' if prediction[0] == 0 else '>50K'}")
if probability is not None:
    print(f"Probability of salary >50K: {probability[0]:.2f}")
else:
    print("Probability not available for this model.")


Predicted salary: >50K
Probability of salary >50K: 0.82
